In [25]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Model training

In [26]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('Large-image-dataset/training',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 2225 images belonging to 2 classes.


In [27]:
print(training_set.classes)
unique, counts = np.unique(training_set.classes, return_counts=True)
print(dict(zip(training_set.class_indices, counts)))

[0 0 0 ... 1 1 1]
{'Legal': 762, 'No ball': 1463}


In [28]:
print(training_set.samples)

2225


In [29]:
from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle
import numpy as np

# Initialize empty lists for training data
X_train = []
y_train = []

# Iterate through the training set and collect the data
i=1
for x, y in training_set:
    print(i)
    i+=1
    X_train.extend(x)
    y_train.extend(y)
    if len(X_train) >= training_set.samples:
        break
# Convert the lists into numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

# Reshape the image data to a 2D array (flatten each image)
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)

# Apply SMOTE to the reshaped training data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_reshaped, y_train)

# Reshape the resampled data back to its original 4D shape (64x64 images with 3 color channels)
X_resampled = X_resampled.reshape(-1, 64, 64, 3)

# Shuffle the resampled data
X_resampled, y_resampled = shuffle(X_resampled, y_resampled, random_state=42)

# Now you have X_resampled and y_resampled with balanced classes


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70


In [30]:
print(X_resampled.shape)
print(y_resampled.shape)

(2926, 64, 64, 3)
(2926,)


In [31]:
new_datagen = ImageDataGenerator(rescale=1./255)

# Create the new training set using the resampled data
training_set_resampled = new_datagen.flow(X_resampled, y_resampled, batch_size=32)
# Convert X_resampled and y_resampled into a TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_resampled, y_resampled))

# Shuffle, batch, and optionally prefetch the data for efficient training
training_set = train_dataset.shuffle(buffer_size=1024).batch(32).prefetch(tf.data.experimental.AUTOTUNE)

# Use this dataset in your model training

In [32]:
print(y_resampled)
unique, counts = np.unique(y_resampled, return_counts=True)
print(dict(zip(unique, counts)))

[0. 1. 1. ... 1. 1. 0.]
{0.0: 1463, 1.0: 1463}


In [33]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('Large-image-dataset/testing',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 262 images belonging to 2 classes.


In [34]:
cnn = tf.keras.models.Sequential()

In [35]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

In [36]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [37]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [38]:
cnn.add(tf.keras.layers.Flatten())

In [39]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [40]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [41]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [42]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 10)

Epoch 1/10
92/92 [==============================] - 15s 126ms/step - loss: 0.5959 - accuracy: 0.6678 - val_loss: 0.5085 - val_accuracy: 0.7748
Epoch 2/10
92/92 [==============================] - 8s 89ms/step - loss: 0.3715 - accuracy: 0.8370 - val_loss: 0.4904 - val_accuracy: 0.8130
Epoch 3/10
92/92 [==============================] - 8s 83ms/step - loss: 0.2444 - accuracy: 0.8978 - val_loss: 0.3628 - val_accuracy: 0.8779
Epoch 4/10
92/92 [==============================] - 7s 81ms/step - loss: 0.1660 - accuracy: 0.9347 - val_loss: 0.3688 - val_accuracy: 0.9160
Epoch 5/10
92/92 [==============================] - 7s 77ms/step - loss: 0.1020 - accuracy: 0.9658 - val_loss: 0.3503 - val_accuracy: 0.9313
Epoch 6/10
92/92 [==============================] - 7s 72ms/step - loss: 0.0650 - accuracy: 0.9781 - val_loss: 0.3263 - val_accuracy: 0.9275
Epoch 7/10
92/92 [==============================] - 7s 73ms/step - loss: 0.0601 - accuracy: 0.9795 - val_loss: 0.4938 - val_accuracy: 0.8931
Epoch 8/10


In [43]:
cnn.save('final_model.keras')

### Model testing

In [44]:
from tensorflow.keras.preprocessing import image
import numpy as np

loaded_model = tf.keras.models.load_model('final_model.keras')

In [45]:
# Load the test image
test_image = image.load_img('test/legal.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

result = loaded_model.predict(test_image)
print(result)

if result[0][0] == 0:
  prediction = 'Legal Ball'
else:
  prediction = 'No Ball'
print(prediction)

1/1 [==============================] - 0s 206ms/step
[[2.475035e-21]]
No Ball


In [46]:
# Load the test image
test_image = image.load_img('test/noball_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

result = loaded_model.predict(test_image)
print(result)

if result[0][0] == 0:
  prediction = 'Legal Ball'
else:
  prediction = 'No Ball'
print(prediction)

1/1 [==============================] - 0s 38ms/step
[[0.]]
Legal Ball


In [47]:
# Load the test image
test_image = image.load_img('test/noball_2.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

result = loaded_model.predict(test_image)
print(result)

if result[0][0] == 0:
  prediction = 'Legal Ball'
else:
  prediction = 'No Ball'
print(prediction)

1/1 [==============================] - 0s 44ms/step
[[1.]]
No Ball


In [48]:
print(tf.version.VERSION)

2.10.0
